In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets tokenizers seqeval -q

In [ ]:
import datasets
train_dataset = datasets.load_dataset('csv', data_files='/content/drive/MyDrive/ResearchTrial/sentence_labels_aptner.csv')
valid_dataset = datasets.load_dataset('csv', data_files='/content/drive/MyDrive/ResearchTrial/sentence_labels_valid_apt.csv')
test_dataset = datasets.load_dataset('csv', data_files='/content/drive/MyDrive/research/sentence_labels_test_apt.csv')

In [ ]:
import datasets
import numpy as np
from transformers import BertTokenizerFast
from transformers import RobertaTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

In [ ]:
# tokenizer = BertTokenizerFast.from_pretrained("CyberPeace-Institute/SecureBERT-NER")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

Align Label


In [ ]:
import ast

def convert_string_to_list(example):
    if isinstance(example["tokens"], str):
        example["tokens"] = ast.literal_eval(example["tokens"])
    if isinstance(example["ner_tags"], str):
        example["ner_tags"] = ast.literal_eval(example["ner_tags"])
    return example

In [ ]:
train_dataset = train_dataset.map(convert_string_to_list )

valid_dataset = valid_dataset.map(convert_string_to_list )

test_dataset = test_dataset.map(convert_string_to_list)



In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    """
    Function to tokenize and align labels with respect to the tokens. This function is specifically designed for
    Named Entity Recognition (NER) tasks where alignment of the labels is necessary after tokenization.

    Parameters:
    examples (dict): A dictionary containing the tokens and the corresponding NER tags.
                     - "tokens": list of words in a sentence.
                     - "ner_tags": list of corresponding entity tags for each word.

    label_all_tokens (bool): A flag to indicate whether all tokens should have labels.
                             If False, only the first token of a word will have a label,
                             the other tokens (subwords) corresponding to the same word will be assigned -100.

    Returns:
    tokenized_inputs (dict): A dictionary containing the tokenized inputs and the corresponding labels aligned with the tokens.
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_valid_datasets = valid_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_test_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)


In [ ]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=85, ignore_mismatched_sizes=True)

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]

In [ ]:
import accelerate

from transformers import TrainingArguments, Trainer
args = TrainingArguments(
"test-ner",
 evaluation_strategy="epoch",
learning_rate=2e-5,
per_device_train_batch_size=8,
per_device_eval_batch_size=8,
num_train_epochs=1,
weight_decay=0.01,
logging_steps=50,
)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = datasets.load_metric("seqeval")

In [ ]:
example = train_dataset['train'][0]

In [ ]:
## FOR DNRTI dataset

# def compute_metrics(eval_preds):
#     """
#     Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
#     The function computes precision, recall, F1 score and accuracy.

#     Parameters:
#     eval_preds (tuple): A tuple containing the predicted logits and the true labels.

#     Returns:
#     A dictionary containing the precision, recall, F1 score and accuracy.
#     """

#     label_list = [
#     'B-HackOrg', 'I-HackOrg',  # Assuming these start from 1
#     'B-OffAct', 'I-OffAct',
#     'B-SamFile', 'I-SamFile',
#     'B-SecTeam', 'I-SecTeam',
#     'B-Tool', 'I-Tool',
#     'B-Time', 'I-Time',
#     'B-Purp', 'I-Purp',
#     'B-Area', 'I-Area',
#     'B-Idus', 'I-Idus',
#     'B-Org', 'I-Org',
#     'B-Way', 'I-Way',
#     'B-Exp', 'I-Exp',
#     'B-Features', 'I-Features','O'
#     # Make sure you have 27 labels here including 'O'
# ]
#     # np.savetxt('eval_preds.txt', eval_preds, delimiter=',')

#     # print(eval_preds)

#     pred_logits, labels = eval_preds

#     pred_logits = np.argmax(pred_logits, axis=2)
#     # the logits and the probabilities are in the same order,
#     # so we don’t need to apply the softmax

#     # We remove all the values where the label is -100
#     predictions = [
#         [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(pred_logits, labels)
#     ]

#     true_labels = [
#       [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
#        for prediction, label in zip(pred_logits, labels)
#    ]
#     results = metric.compute(predictions=predictions, references=true_labels)
#     return {
#    "precision": results["overall_precision"],
#    "recall": results["overall_recall"],
#    "f1": results["overall_f1"],
#   "accuracy": results["overall_accuracy"],
#   }

In [ ]:
## For APTNER dataset

def compute_metrics(eval_preds):


    label_list = [
    'B-APT', 'I-APT', 'E-APT', 'S-APT',
    'B-SECTEAM', 'I-SECTEAM', 'E-SECTEAM', 'S-SECTEAM',
    'B-IDTY', 'I-IDTY', 'E-IDTY', 'S-IDTY',
    'B-OS', 'I-OS', 'E-OS', 'S-OS',
    'B-EMAIL', 'I-EMAIL', 'E-EMAIL', 'S-EMAIL',
    'B-LOC', 'I-LOC', 'E-LOC', 'S-LOC',
    'B-TIME', 'I-TIME', 'E-TIME', 'S-TIME',
    'B-IP', 'I-IP', 'E-IP', 'S-IP',
    'B-DOM', 'I-DOM', 'E-DOM', 'S-DOM',
    'B-URL', 'I-URL', 'E-URL', 'S-URL',
    'B-PROT', 'I-PROT', 'E-PROT', 'S-PROT',
    'B-FILE', 'I-FILE', 'E-FILE', 'S-FILE',
    'B-TOOL', 'I-TOOL', 'E-TOOL', 'S-TOOL',
    'B-MD5', 'I-MD5', 'E-MD5', 'S-MD5',
    'B-SHA1', 'I-SHA1', 'E-SHA1', 'S-SHA1',
    'B-SHA2', 'I-SHA2', 'E-SHA2', 'S-SHA2',
    'B-MAL', 'I-MAL', 'E-MAL', 'S-MAL',
    'B-ENCR', 'I-ENCR', 'E-ENCR', 'S-ENCR',
    'B-VULNAME', 'I-VULNAME', 'E-VULNAME', 'S-VULNAME',
    'B-VULID', 'I-VULID', 'E-VULID', 'S-VULID',
    'B-ACT', 'I-ACT', 'E-ACT', 'S-ACT', 'O'
    ]


    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)

    # We remove all the values where the label is -100
    predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
      "precision": results["overall_precision"],
      "recall": results["overall_recall"],
      "f1": results["overall_f1"],
      "accuracy": results["overall_accuracy"],
      }

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/ResearchTrial/SECOND_ATTEMPTS/bertcasedapt/ner_model_1")

In [ ]:
trainer = Trainer(
    model,
    args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_valid_datasets["train"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
label_names = [
    'B-HackOrg', 'I-HackOrg',  # Assuming these start from 1
    'B-OffAct', 'I-OffAct',
    'B-SamFile', 'I-SamFile',
    'B-SecTeam', 'I-SecTeam',
    'B-Tool', 'I-Tool',
    'B-Time', 'I-Time',
    'B-Purp', 'I-Purp',
    'B-Area', 'I-Area',
    'B-Idus', 'I-Idus',
    'B-Org', 'I-Org',
    'B-Way', 'I-Way',
    'B-Exp', 'I-Exp',
    'B-Features', 'I-Features','O'
    # Make sure you have 27 labels here including 'O'
]

In [ ]:
### Metrics for test set

predictions, labels, _ = trainer.predict(tokenized_test_datasets["train"])
predictions = np.argmax(predictions, axis=2)
# Remove ignored index (special tokens)
true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
### Metrics for train set

predictions, labels, _ = trainer.predict(tokenized_datasets["train"])
predictions = np.argmax(predictions, axis=2)
# Remove ignored index (special tokens)
true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
# model.save_pretrained("/content/drive/MyDrive/ResearchTrial/BertBaseNERApt/ner_model_5")